In [7]:
import sys
print(sys.version)

3.10.5 (tags/v3.10.5:f377153, Jun  6 2022, 16:14:13) [MSC v.1929 64 bit (AMD64)]


fastapi的核心功能是提供HTTP请求接口。在了解HTTP请求方式之前，我们先来了解一下“幂等”和“非幂等”。

幂等性，英文是idempotent，读作[a'dmptnt]。它的含义如下：

“Methods can also have the property of "idempotence" in that the side-effects of N > 0 identical requests is the same as for a single request.”

（这句话翻译过来是这样的：方法可以有幂等性，幂等性指的是N>0次的完全相同的请求的副作用和一次单个请求的副作用是相同的）。

即，如果一个方法重复执行多次，产生的效果是一样的，那么这个方法就是幂等的。

最常见的HTTP请求方法是四个：GET、DELETE、POST、PUT。幂等性列举如下：

| 请求方法 | 含义 | 幂等性 |  
| :---: | :---: | :---: |  
| GET | 用于请求指定的资源 | 是 |  
| DELETE | 用于删除指定的资源 | 是 |  
| POST | 用于向指定资源提交数据，数据被包含在请求体中 | 否 |  
| PUT | 用于向指定资源位置上传新的内容，如果该资源已经存在则会被覆盖 



GET 请求可被缓存；GET 请求保留在浏览器历史记录中；GET 请求可被收藏为书签；GET 请求不应在处理敏感数据时使用；GET 请求有长度限制；GET 请求只应当用于取回数据。


POST 请求不会被缓存；POST 请求不会保留在浏览器历史记录中；POST 不能被收藏为书签；POST 请求对数据长度没有要求

POST和PUT的区别

（1） POST是用来提交数据的。提交的数据放在HTTP请求的正文里，目的在于提交数据并用于服务器端的存储，而不允许用户过多的更改相应数据（主要是相对于在url 修改要麻烦很多)
。
（2） PUT操作是幂等的。所谓幂等是指不管进行多少次操作，结果都一样。比如我用PUT修改一篇文章，然后在做同样的操作，每次操作后的资源整体结果并没有不同（文章具体内容可能有变化，但文章整体作为资源标识并没有变化
）。
（3）POST操作既不是安全的，也不是幂等的，比如常见的POST重复加载问题：当我们多次发出同样的POST请求后，其结果是创建出了若干的
资源。
（4）安全和幂等的意义在于：当操作没有达到预期的目标时，我们可以不停的重试，而不会对资源产生副作用。从这个意义上说，POST操作往往是有害的，但很多时候我们还是不得不
使用它。
（5）创建操作可以使用POST，也可以使用PUT，区别在于POST 是作用在一个集合资源之上的（/articles），而PUT操作是作用在一个具体资源之上的（/articles/id123）；再通俗点说，如果URL可以在客户端确定，那么就使用PUT，如果是在服务端确定，那么就使用POST，比如说很多资源使用数据库自增主键作为标识信息，而创建的资源的标识信息到底是什么只能由服务端提供，这个时候就必须使用POST。。| 是 |

In [6]:
import uvicorn
from fastapi import FastAPI
app = FastAPI()
@app.get("/")
async def root():
    return {"message": "Hello World"}

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [16524]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8009 (Press CTRL+C to quit)


INFO:     127.0.0.1:56730 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:56730 - "GET /favicon.ico HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [16524]


查看  
打开浏览器访问 http://127.0.0.1:8000  。

你将看到如下的 JSON 
```python
{"message": "Hello World"}
```
交互式 API 文档  
跳转到 http://127.0.0.1:8000/docs。

你将会看到自动生成的交互式 API 文档（由 Swagger UI 提供）：响应：

在开发 API 时，你通常使用特定的 HTTP 方法去执行特定的行为。

通常使用：

POST：创
建数据。
GET：
读取数据。
PUT
：更新数据。
DELET
E：删除数据。
因此，在 OpenAPI 中，每一个 HTTP 方法都被称为「操作」。

我们也打算称呼它们为「操作」。

## 路径参数

In [8]:
import uvicorn
from fastapi import FastAPI
app = FastAPI()
@app.get("/items/{item_id}")
async def read_item(item_id: int):
    return {"item_id": item_id}

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8009)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [16524]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8009 (Press CTRL+C to quit)


INFO:     127.0.0.1:53974 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:53975 - "GET /items/3 HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [16524]


浏览器中打开http://127.0.0.1:8009/items/3

会看到{"item_id":3}

路径参数 item_id 的值将作为参数 item_id 传递给你的函数。

item_id 被声明为 int 类型。

这将为你的函数提供编辑器支持，包括错误检查、代码补全等等。

但如果你通过浏览器访问 http://127.0.0.1:8009/items/foo，你会看到一个清晰可读的 HTTP 错误

如果你提供的是 float 而非整数也会出现同样的错误，比如： http://127.0.0.1:8009/items/4.2

## Pydantic

所有的数据校验都由 Pydantic 在幕后完成，所以你可以从它所有的优点中受益。并且你知道它在这方面非常胜任。

你可以使用同样的类型声明来声明 str、float、bool 以及许多其他的复合数据类型。

## 预设值

如果你有一个接收路径参数的路径操作，但你希望预先设定可能的有效参数值，则可以使用标准的 Python Enum 类型。

创建一个 Enum 类
导入 Enum 并创建一个继承自 str 和 Enum 的子类。

通过从 str 继承，API 文档将能够知道这些值必须为 string 类型并且能够正确地展示出来。

然后创建具有固定值class ModelName(str, Enum)，的类属性，这些固定值将是可

然后使用你定义的枚举类（ModelName）创建一个带有类型标注的路径参数async def get_model(model_name: ModelName)的有效值：

In [11]:
import uvicorn
from fastapi import FastAPI
from enum import Enum

class ModelName(str, Enum):
    alexnet = "alexnet"
    resnet = "resnet"
    lenet = "lenet"

app = FastAPI()

@app.get("/models/{model_name}")
async def get_model(model_name: ModelName):
    if model_name is ModelName.alexnet:
        return {"model_name": model_name, "message": "Deep Learning FTW!"}

    if model_name.value == "lenet":
        return {"model_name": model_name, "message": "LeCNN all the images"}

    return {"model_name": model_name, "message": "Have some residuals"}

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8000)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [16524]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:54160 - "GET /models/resnet HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [16524]


在浏览器中输入http://127.0.0.1:8000/models/resnet

会看到：
{"model_name":"resnet","message":"Have some residuals"}

models/ 后面只能是'alexnet', 'resnet' or 'lenet'，如果输入其他字符，会显示
{"detail":[{"type":"enum","loc":["path","model_name"],"msg":"Input should be 'alexnet', 'resnet' or 'lenet'","input":"resne","ctx":{"expected":"'alexnet', 'resnet' or 'lenet'"}}]}

## 获取枚举值

你可以使用 model_name.value 或通常来说 your_enum_member.value 来获取实际的值（在这个例子中为 str）

## 包含路径的路径参数

假设你有一个路径操作，它的路径为 /files/{file_path}。

但是你需要 file_path 自身也包含路径，比如 home/johndoe/myfile.txt。

因此，该文件的URL将类似于这样：/files/home/johndoe/myfile.txt。

你可以使用直接来自 Starlette 的选项来声明一个包含路径的路径参数：

/files/{file_path:path}

在这种情况下，参数的名称为 file_path，结尾部分的 :path 说明该参数应匹配任意的路径。

因此，你可以这样使用它：

In [ ]:
import uvicorn
from fastapi import FastAPI
app = FastAPI()
@app.get("/files/{file_path:path}")
async def read_file(file_path: str):
    return {"file_path": file_path}

if __name__ == '__main__':
    config = uvicorn.Config(app, host='0.0.0.0', port=8000)
    server = uvicorn.Server(config)
    await server.serve()

INFO:     Started server process [16524]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:54386 - "GET /files/home/johndoe/myfile.txt HTTP/1.1" 200 OK
INFO:     127.0.0.1:54398 - "GET /files//home/johndoe/myfile.txt HTTP/1.1" 200 OK
INFO:     127.0.0.1:54398 - "GET /files//home/johndoe/myfile.txt HTTP/1.1" 200 OK


打开浏览器，输入 http://127.0.0.1:8000/files/home/johndoe/myfile.txt

会看到：
{"file_path":"home/johndoe/myfile.txt"}